# Исследование текущего уровня потребительской лояльности (NPS) среди клиентов из России

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

**Цель исследования** - построить дашборд, позволяющий  определить текущий уровень потребительской лояльности, или NPS, среди различных клиентов из России. 

**Задачами исследования** являются ответы на следующие вопросы:
- Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе сторонников?


In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
path_to_db = 'telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)


In [3]:
query = """
SELECT u.user_id, u.lt_day,
    CASE 
        WHEN lt_day < 365 THEN 'True'
        ELSE 'False' END AS is_new,
    u.age,
    CASE 
        WHEN u.gender_segment = 0 THEN 'мужчина'
        ELSE 'женщина' END AS gender_segment,
    u.os_name, u.cpe_type_name, l.country, l.city, SUBSTR(ags.title,3) as age_segment, SUBSTR(ts.title,3) as traffic_segment,
    SUBSTR(ls.title,3) as lifetime_segment, u.nps_score,
    CASE 
        WHEN u.nps_score <= 6 THEN 'критики'
        WHEN u.nps_score <= 8 THEN 'нейтралы'
        ELSE 'cторонники' END AS nps_group
FROM user AS u
LEFT JOIN location l ON u.location_id = l.location_id
LEFT JOIN age_segment ags ON u.age_gr_id = ags.age_gr_id
LEFT JOIN traffic_segment ts ON u.tr_gr_id = ts.tr_gr_id
LEFT JOIN lifetime_segment ls ON u.lt_gr_id = ls.lt_gr_id
WHERE (u.age IS NOT NULL) AND (u.gender_segment IS NOT NULL)
"""

#Так как в столбце `gender_segment`и `age` были обнаружены пропуски, стоит их убрать в запросе

In [4]:
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,False,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,False,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,False,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,False,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Выводы:
1. Среди участников опроса превалируют старые клиенты женского пола из Москвы, СПБ, Новосибирска и Екатеринбурга, в возрасте от 25 до 44 лет
2. Более половины опрошенных лояльны к сервису (52,6%). Общий уровень потребительской лояльности (NPS) составляет около 22%
3. Наиболее лояльными к сервису являются новые клиенты, их NPS во всех возрастных сегментах выше, чем у старых клиентов. Лица старших возрастов (от 55 лет и старше) больше всего лояльны к компании. Среди старых клиентов в возрасте до 24 лет NPS отрицательный. Наибольший уровень потребительской лояльности наблюдается в городах Иваново, Череповец и Саранск
4. Среди сторонников превалируют старые клиенты женского пола из Москвы, СПБ, Новосибирска и Казани, в возрасте от 25 до 44 лет. 


### Cсылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/.71623239/viz/2_16712777917320/NPS?publish=yes

### Ссылка на pdf-файл с презентацией:

https://www.dropbox.com/s/2fnz8m7l0tcnj8y/%D0%A1%D0%B1%D0%BE%D1%80%D0%BD%D1%8B%D0%B9%20%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%202.pdf?dl=0